In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
df=pd.read_csv('/content/spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.Category.value_counts()

,count
Category,
ham,4825
spam,747


In [4]:
df_ham=df[df['Category'] == 'ham']
df_ham

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...
...,...,...
5565,ham,Huh y lei...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
df_spam=df[df['Category'] == 'spam']

In [6]:
df_spam

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [7]:
df_ham_downsampled=df_ham.sample(747)

In [8]:
df_ham_downsampled

,Category,Message
1038,ham,"naughty little thought: 'its better to flirt, ..."
170,ham,Yes :)it completely in out of form:)clark also...
4017,ham,In which place i can get rooms cheap:-)
5256,ham,Well she's in for a big surprise!
4369,ham,1 I don't have her number and 2 its gonna be a...
...,...,...
4464,ham,He said i look pretty wif long hair wat. But i...
3831,ham,I agree. So i can stop thinkin about ipad. Can...
102,ham,I'm still looking for a car to buy. And have n...
85,ham,"I call you later, don't have network. If urgnt..."


In [9]:
df_balanced=pd.concat([df_ham_downsampled,df_spam])

In [10]:
df_balanced

,Category,Message
1038,ham,"naughty little thought: 'its better to flirt, ..."
170,ham,Yes :)it completely in out of form:)clark also...
4017,ham,In which place i can get rooms cheap:-)
5256,ham,Well she's in for a big surprise!
4369,ham,1 I don't have her number and 2 its gonna be a...
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [11]:
df_balanced['Category']=df_balanced['Category'].map({'ham': 0,'spam':1})

In [12]:
df_balanced

,Category,Message
1038,0,"naughty little thought: 'its better to flirt, ..."
170,0,Yes :)it completely in out of form:)clark also...
4017,0,In which place i can get rooms cheap:-)
5256,0,Well she's in for a big surprise!
4369,0,1 I don't have her number and 2 its gonna be a...
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_balanced['Message'],df_balanced['Category'],test_size=0.2,stratify=df_balanced['Category'])

In [14]:
y_train

,Category
511,0
1122,1
1142,1
1036,0
134,1
...,...
2868,0
3299,1
1970,1
3226,0


In [15]:
bert_preprocessing=hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
bert_encoder=hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-10-h-128-a-2/2")

In [17]:
preprocessed_text=bert_preprocessing(x_train)
bert_embedding=bert_encoder(preprocessed_text)

In [18]:
model=keras.Sequential([
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(bert_embedding['pooled_output'],y_train,epochs=5)

Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7718 - loss: 0.5222
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8753 - loss: 0.3173
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8956 - loss: 0.2572
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8960 - loss: 0.2691
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9114 - loss: 0.2315


In [19]:
preprocessed_text=bert_preprocessing(x_test)
bert_embedding=bert_encoder(preprocessed_text)
model.evaluate(bert_embedding['pooled_output'],y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9086 - loss: 0.2415  


[0.23317739367485046, 0.9063544869422913]

In [20]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]

preprocessed_text=bert_preprocessing(reviews)
bert_embedding=bert_encoder(preprocessed_text)
model.predict(bert_embedding['pooled_output'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[0.6772519 ],
       [0.9385057 ],
       [0.7392895 ],
       [0.23201312],
       [0.01248393]], dtype=float32)

In [26]:
model.export('/content/My_Model/2')

Saved artifact at '/content/My_Model/2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136935714171856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136935714174352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136935714174160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136935714169552: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [30]:
! zip -r models.zip /content/My_Model

  adding: content/My_Model/ (stored 0%)
  adding: content/My_Model/1/ (stored 0%)
  adding: content/My_Model/1/saved_model.pb (deflated 82%)
  adding: content/My_Model/1/assets/ (stored 0%)
  adding: content/My_Model/1/fingerprint.pb (stored 0%)
  adding: content/My_Model/1/variables/ (stored 0%)
  adding: content/My_Model/1/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/My_Model/1/variables/variables.index (deflated 55%)
  adding: content/My_Model/2/ (stored 0%)
  adding: content/My_Model/2/saved_model.pb (deflated 82%)
  adding: content/My_Model/2/assets/ (stored 0%)
  adding: content/My_Model/2/fingerprint.pb (stored 0%)
  adding: content/My_Model/2/variables/ (stored 0%)
  adding: content/My_Model/2/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/My_Model/2/variables/variables.index (deflated 55%)
